![ros_control_logo](images/logos/header_logos.png)

![](images/logos/image_ROSDevDay21.png)

# From ros_control to ros2_control #

A brief, hands-on introduction to ros_control.
We will use ROS Noetic to show you
* when and how to use ros_control,
* how to implement a Hardware Abstraction Layer (HAL),
* how to implement a controller for your ros_control setup.

Building on your newfound ros_control knowledge, we move to ROS2 Foxy and

* introduce you to the concepts of ros2_control,
* show you how to implement a System component (sub-HAL),
* demonstrate how to implement a controller for a ros2_control setup.

Buckle up, there's a lot to learn today!

We are going to use a 2 degrees of freedom robot arm for demoing. 
![rrbot](images/rrbot.png)

But here's a *small* list of robots that use ros_control
![ROS1 ros_control robots](images/ros_control_montage.jpg)

## WHO ARE WE

### Bence Magyar, PhD in Robotics, Lead Software Engineer at FiveAI <div style="width: 100px;">![bence_image](images/bence.jpeg)</div> 
maintainer at ros_control and ros2_control

### ‪Denis Štogl, PhD in Robotics, Robotics Consultant at Stogl Robotics <div style="width: 100px;">![denis_image](images/denis.jpeg)</div> 
maintainer at ros2_control


## ros_control in a nutshell

![](images/overview.png)

* RobotHW is also referred to as the Hardware Abstraction Layer (HAL).
* Controllers claim resources from controller manager (safe)
* Controllers write set commands on claimed joint interface handles


![interfaces_and_controllers](images/roscontrol_interfaces_controllers.png)

Today we'll explain you how to write a RobotHW and a controller and what steps are important to pay attention to.

For the first set of demos we are going to use the ROS Noetic environment. Make sure to source first.

```
cd catkin_ws && source /opt/ros/noetic/setup.bash
catkin_make 
```


```
source catkin_ws/devel/setup.bash
```

Let's take a look at the robot we'll use!
```
roslaunch rrbot_description view_robot.launch
```

## Implementing a ros_control RobotHW 

The general structure of RobotHW is as follows:
* load URDF
* establish controller_manager service interfaces
* main loop
 * read()
 * update() -> call update() of controllers
 * write()

The same as C++:
DENIS INSERT HEADER CODE HERE

INSERT CPP CODE HERE (EXTRACT)

LAUNCH ROBOTHW WITH STATE PUBLISHER AND ECHO SOMETHING BY HAND

## Implementing a controller for ros_control

Let's take a look at how a controller is structured.

ADD SOME C++ EXAMPLE

`roslaunch rrbot_bringup rrbot_with_rrbot_controller_array.launch`

ADD COMMAND


ADD C++ CHANGES

`roslaunch rrbot_bringup rrbot_with_rrbot_controller.launch`

ADD COMMAND

But we can achieve this with a standard forward command controller:

```roslaunch rrbot_bringup rrbot.launch```

ADD COMMAND

# ROS2 and ros\_control = ros2\_control

In ros2_control, the functionalities of a HAL and RobotHW are separated. 

The role of HAL is taken by the ResourceManager which can manage multiple Sensor, System or Actuator components. 
These components serve as self-contained hardware drivers, essentially equivalent to ros_control's RobotHW.

![](images/components_architecture.png)


For the following demos we'll use the ROS2 Foxy environment, make sure to run

Checking rrbot

```
ros2 launch rrbot_description view_robot.launch.py
```

![rrbot rviz](images/rrbot_gui.png)

## Implementing a system component in ros2_control

The integration of `ros2_control` is done through the `ros2_control` URDF tag

```xml
<ros2_control name="${name}" type="system">
      <hardware>
          <plugin>fake_components/GenericSystem</plugin>
      </hardware>
      <joint name="joint1">
        <command_interface name="position">
          <param name="min">-1</param>
          <param name="max">1</param>
        </command_interface>
        <state_interface name="position"/>
      </joint>
      <joint name="joint2">
        <command_interface name="position">
          <param name="min">-1</param>
          <param name="max">1</param>
        </command_interface>
        <state_interface name="position"/>
      </joint>
    </ros2_control>
```

which will pick our pre-supplied implementation of a system component that always "simulates" perfect execution.

Let's write one!

```C++
class RRBotHardwareInterface
  : public hardware_interface::BaseInterface<hardware_interface::SystemInterface>
{
public:
  RCLCPP_SHARED_PTR_DEFINITIONS(RRBotHardwareInterface)

  RRBOT_HARDWARE_INTERFACE_PUBLIC
  hardware_interface::return_type configure(const hardware_interface::HardwareInfo & info) override;

  RRBOT_HARDWARE_INTERFACE_PUBLIC
  std::vector<hardware_interface::StateInterface> export_state_interfaces() override;

  RRBOT_HARDWARE_INTERFACE_PUBLIC
  std::vector<hardware_interface::CommandInterface> export_command_interfaces() override;

  RRBOT_HARDWARE_INTERFACE_PUBLIC
  hardware_interface::return_type start() override;

  RRBOT_HARDWARE_INTERFACE_PUBLIC
  hardware_interface::return_type stop() override;

  RRBOT_HARDWARE_INTERFACE_PUBLIC
  hardware_interface::return_type read() override;

  RRBOT_HARDWARE_INTERFACE_PUBLIC
  hardware_interface::return_type write() override;

private:
  std::vector<double> hw_commands_;
  std::vector<double> hw_states_;
};
```

while in the cpp file...

```C++
hardware_interface::return_type RRBotHardwareInterface::read()
{
  // read robot states from hardware, in this example print only
  RCLCPP_INFO(rclcpp::get_logger("RRBotHardwareInterface"), "Reading...");

  // write command to hardware, in this example do mirror command to states
  for (size_t i = 0; i < hw_states_.size(); ++i){
    RCLCPP_INFO(
      rclcpp::get_logger("RRBotHardwareInterface"),
      "Got state %.2f for joint %d!", hw_states_[i], i);
  }

  return hardware_interface::return_type::OK;
}

hardware_interface::return_type RRBotHardwareInterface::write()
{
  // write command to hardware, in this example do mirror command to states
  for (size_t i = 0; i < hw_commands_.size(); ++i){
    hw_states_[i] = hw_commands_[i];
  }

  return hardware_interface::return_type::OK;
}
```

The updated ros2_control tag will look like this:
```xml
<ros2_control name="${name}" type="system">
      <hardware> 
        <plugin>rrbot_hardware_interface/RRBotHardwareInterface</plugin>
      </hardware>
      <joint name="joint1">
        <command_interface name="position">
          <param name="min">-1</param>
          <param name="max">1</param>
        </command_interface>
        <state_interface name="position"/>
      </joint>
      <joint name="joint2">
        <command_interface name="position">
          <param name="min">-1</param>
          <param name="max">1</param>
        </command_interface>
        <state_interface name="position"/>
      </joint>
</ros2_control>
```

## Implementing an array-based controller in ros2_control

```C++
class RRBotControllerArray : public controller_interface::ControllerInterface
{
public:
  controller_interface::return_type init(const std::string & controller_name) override;

  controller_interface::InterfaceConfiguration command_interface_configuration() const override;

  controller_interface::InterfaceConfiguration state_interface_configuration() const override;

  CallbackReturn on_configure(const rclcpp_lifecycle::State & previous_state) override;

  CallbackReturn on_activate(const rclcpp_lifecycle::State & previous_state) override;

  CallbackReturn on_deactivate(const rclcpp_lifecycle::State & previous_state) override;

  controller_interface::return_type update() override;

protected:
  std::vector<std::string> joint_names_;
  std::string interface_name_;

  using ControllerCommandMsg = example_interfaces::msg::Float64MultiArray;

  rclcpp::Subscription<ControllerCommandMsg>::SharedPtr command_subscriber_ = nullptr;
  realtime_tools::RealtimeBuffer<std::shared_ptr<ControllerCommandMsg>> input_command_;

  using ControllerStateMsg = control_msgs::msg::JointControllerState;
  using ControllerStatePublisher = realtime_tools::RealtimePublisher<ControllerStateMsg>;

  rclcpp::Publisher<ControllerStateMsg>::SharedPtr s_publisher_;
  std::unique_ptr<ControllerStatePublisher> state_publisher_;
};
```

Bring up ros2_control with the array-based controller running:
```
ros2 launch rrbot_bringup rrbot_with_rrbot_controller_array.launch.py
```

Let's test the controller by sending it some commands!

```
ros2 topic pub /rrbot_controller/commands example_interfaces/msg/Float64MultiArray "data:
- 0.5
- 0.5" 
```

## Implementing a "jog" controller in ros2_control

Let's replace Float64MultiArray with something more specific, such as JointJog. 

TODO SHOW MESSAGE LAYOUTS HERE

Modifying the code of the previous controller, simply change the `ControllerCommandMsg` type in the header

```C++
class RRBotController : public controller_interface::ControllerInterface
{
public:
  ...
protected:
  ...
  
  using ControllerCommandMsg = control_msgs::msg::JointJog;

  ...
};
```

...and re-adjust some of the implementation reference from `data` to `displacement`.

Testing the new controller: add controller and configuration file to launch file

`ros2 launch rrbot_bringup rrbot_with_rrbot_controller.launch.py`

Check which controllers are loaded now and which interfaces are claimed

```
ros2 control list_controllers
```

```
ros2 control list_hardware_interfaces
```

Examine the state of the controller on the state topic or the joint states topic

```
ros2 topic echo /rrbot_controller/state
```


```
ros2 topic echo /joint_states
```

```
ros2 topic pub /rrbot_controller/commands control_msgs/msg/JointJog "joint_names:
- joint1
- joint2
displacements:
- 0.5
- 0.5"
```

## [Concluding remarks]


[Write here your conclusions]

# References

* [ros_control paper in the Journal of Open Source Software](https://joss.theoj.org/papers/10.21105/joss.00456)
* ros_control
 * http://wiki.ros.org/ros_control
 * http://wiki.ros.org/ros_controllers (select individual controllers) 
 * https://github.com/ros-controls/roadmap/blob/master /documentation_resources.md
* ros2_control
 * https://ros-controls.github.io/control.ros.org/
 * https://github.com/ros-controls/ros2_control
 * https://github.com/ros-controls/ros2_controllers
 * https://github.com/ros-controls/ros2_control_demos

# Extra ROSject Documentation

All source code for ros_control on ROS Noetic can be found in `catkin_ws` while all resources for ros2_control on ROS Foxy is located in `ros2_ws`.